In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


Processando o texto

Vectorize o texto
Antes do treinamento, precisamos mapear strings para uma representação numérica. Crie duas tabelas de pesquisa: uma mapeando caracteres para números e outra para números para caracteres.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Agora temos uma representação inteira para cada personagem. Observe que mapeamos o caractere como índices de 0 a len(unique) .



In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


previsão

In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


O método em batch nos permite converter facilmente esses caracteres individuais em sequências do tamanho desejado.



In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para cada sequência, duplique e mude para formar o texto de entrada e de destino usando o método de map para aplicar uma função simples a cada lote:



In [12]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Imprima os primeiros exemplos de valores de entrada e destino:



In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Cada índice desses vetores é processado como uma etapa de tempo. Para a entrada na etapa de tempo 0, o modelo recebe o índice para "F" e tenta prever o índice para "i" como o próximo caractere. No próximo passo de tempo, ele faz a mesma coisa, mas o RNN considera o contexto da etapa anterior além do caractere de entrada atual.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


**Cria treinamentos em lote**


Usamos tf.data para dividir o texto em sequências gerenciáveis. Mas antes de alimentar esses dados no modelo, precisamos embaralhar os dados e empacotá-los em lotes.




In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

**Construa o modelo**

Use tf.keras.Sequential para definir o modelo. Para este exemplo simples, três camadas são usadas para definir nosso modelo:

tf.keras.layers.Embedding : A camada de entrada. Uma tabela de pesquisa treinável que mapeará os números de cada caractere para um vetor com dimensões embedding_dim ;
tf.keras.layers.GRU : Um tipo de RNN com units=rnn_units tamanho units=rnn_units (você também pode usar uma camada LSTM aqui.)
tf.keras.layers.Dense : A camada de saída, com saídas vocab_size .

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Imagem das rede

Experimente o modelo
Agora execute o modelo para ver se ele se comporta conforme o esperado.

Primeiro verifique a forma da saída:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


No exemplo acima, o comprimento da sequência da entrada é 100 mas o modelo pode ser executado em entradas de qualquer comprimento:

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


Para obter previsões reais do modelo, precisamos obter uma amostra da distribuição de saída para obter índices de caracteres reais. Esta distribuição é definida pelos logits sobre o vocabulário dos caracteres.

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Isso nos dá, a cada passo de tempo, uma previsão do próximo índice de caractere:

In [22]:
sampled_indices

array([12, 38,  8, 35, 51, 57, 18, 15, 16, 61, 27, 22,  9, 52, 30, 26,  0,
       53,  7, 53, 24, 63, 26,  3, 35,  0,  9, 63, 41, 50, 57, 42, 21, 27,
       35,  8, 26, 18, 32, 36, 39,  4,  6, 50, 37, 41, 57, 13, 22, 33, 44,
        1, 28, 18, 24, 12, 22,  6, 26, 24, 63, 39, 20, 44,  3, 59,  0, 63,
       12,  2, 45, 14, 27, 43, 27, 30, 25, 32, 37, 24, 49, 39, 51, 38, 48,
        4, 15, 60, 63, 12, 45,  6, 40, 58, 52, 56, 60, 63, 32, 42])

Decodifique-os para ver o texto previsto por este modelo não treinado:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' seen him thus.\n\nMARCIUS:\n\nCOMINIUS:\nThe shepherd knows not thunder from a tabour\nMore than I know t'

Next Char Predictions: 
 '?Z.WmsFCDwOJ3nRN\no-oLyN$W\n3yclsdIOW.NFTXa&,lYcsAJUf PFL?J,NLyaHf$u\ny?!gBOeORMTYLkamZj&Cvy?g,btnrvyTd'


**Treine o modelo**

Neste ponto, o problema pode ser tratado como um problema de classificação padrão. Dado o estado RNN anterior e a entrada desta etapa de tempo, preveja a classe do próximo caractere.

Anexe um otimizador e uma função de perda
A função de perda tf.keras.losses.sparse_categorical_crossentropy padrão funciona neste caso porque é aplicada na última dimensão das previsões.

Como nosso modelo retorna logits, precisamos definir o sinalizador from_logits .

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1750736


Configure o procedimento de treinamento usando o método tf.keras.Model.compile . Usaremos tf.keras.optimizers.Adam com argumentos padrão e a função de perda.

In [25]:
model.compile(optimizer='adam', loss=loss)

**Configurar pontos de verificação**

Use um tf.keras.callbacks.ModelCheckpoint para garantir que os pontos de verificação sejam salvos durante o treinamento:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Execute o treinamento**

Para manter o tempo de treinamento razoável, use 10 épocas para treinar o modelo. No Colab, defina o tempo de execução para GPU para um treinamento mais rápido.

In [27]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 903s 5s/step - loss: 2.6196
Epoch 2/10
172/172 [==============================] - 864s 5s/step - loss: 1.9394
Epoch 3/10
172/172 [==============================] - 873s 5s/step - loss: 1.6749
Epoch 4/10
172/172 [==============================] - 871s 5s/step - loss: 1.5320
Epoch 5/10
172/172 [==============================] - 868s 5s/step - loss: 1.4468
Epoch 6/10
172/172 [==============================] - 861s 5s/step - loss: 1.3883
Epoch 7/10
172/172 [==============================] - 875s 5s/step - loss: 1.3434
Epoch 8/10
172/172 [==============================] - 866s 5s/step - loss: 1.3054
Epoch 9/10
172/172 [==============================] - 864s 5s/step - loss: 1.2702
Epoch 10/10
172/172 [==============================] - 859s 5s/step - loss: 1.2379


**Gerar texto **

Restaure o último ponto de verificação
Para manter essa etapa de previsão simples, use um tamanho de lote de 1.

Por causa da forma como o estado RNN é passado de timestep para timestep, o modelo só aceita um tamanho de lote fixo depois de construído.

Para executar o modelo com um batch_size diferente, precisamos reconstruir o modelo e restaurar os pesos do ponto de verificação.

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


**O loop de previsão **

O seguinte bloco de código gera o texto:

Ele começa escolhendo uma string inicial, inicializando o estado RNN e definindo o número de caracteres a serem gerados.

Obtenha a distribuição de previsão do próximo caractere usando a string inicial e o estado RNN.

Em seguida, use uma distribuição categórica para calcular o índice do caráter previsto. Use este caractere previsto como nossa próxima entrada para o modelo.

O estado RNN retornado pelo modelo é realimentado no modelo para que agora tenha mais contexto, em vez de apenas um caractere. Depois de prever o próximo caractere, os estados RNN modificados são novamente realimentados no modelo, que é como ele aprende conforme obtém mais contexto dos caracteres preditos anteriormente.

Imagem aqui

Olhando para o texto gerado, você verá que o modelo sabe quando capitalizar, fazer parágrafos e imitar um vocabulário de escrita semelhante ao de Shakespeare. Com o pequeno número de épocas de treinamento, ainda não aprendeu a formar frases coerentes.

In [31]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I must go with me! were he hat
The more, that I am so but beauty oath.

JULIET:
The matter that our chait of what I love
Much money than shape, two stops
That she had there be at them that I say?
Tell you, pribry that be? speak you, speak be?
Have bee babe the ground of it? Thus ballads, my mistress, which lies
That wench it, that lives long, slaud,
I'come not some on pault.

LADY GREY:
His special apparelling fall about
So she and unreqoice and tenter by the war
Than dies it with your poison's land's change,
To thee favoury that as engrail.

NORTHUMBERLAND:
Welcome! she is your brother, unleantoming
As this is it then appreated: pardon,
I long ip us both; say you go't before 't:
And that this conference 'scape,
And the racime he comes company,
As shall cheekself, thou hadst not so?

TRANIO:
And to lenging beating not? this is impedience
phased for my pittenly guides them not,
For one their beaves him, let not the basif eas! What will our cast wrongs' black land.
What's the god 

A coisa mais fácil que você pode fazer para melhorar os resultados é treiná-lo por mais tempo (tente EPOCHS=30 ).

Você também pode experimentar uma sequência de início diferente ou tentar adicionar outra camada RNN para melhorar a precisão do modelo ou ajustar o parâmetro de temperatura para gerar previsões mais ou menos aleatórias.